In [3]:
from __future__ import print_function

import functools
import threading
import matplotlib.pyplot as plt
import numpy as np
import time
from pyNN.utility.plotting import Figure, Panel

import spynnaker8 as p
from spinn_gym.games.breakout.breakout_sim import (
    get_scores, start_external_visualiser)
from spinn_gym.games.breakout.automated_breakout import (
    AutomatedBreakout, X_RES, X_SCALE, Y_RES, Y_SCALE)
from spinn_gym.games.breakout.visualiser.visualiser import Visualiser
from spinn_front_end_common.utilities.globals_variables import get_simulator

from IPython import display

def start_vis_thread(database, pop_label, xr, yr, xb=8, yb=8, key_conn=None):
    global board_address
    global tag
    
    print("start_vis_thread")
    
    _, _, _, board_address, tag = database.get_live_output_details(
        pop_label, "LiveSpikeReceiver")
    
    print("board_address, tag: ", board_address, tag)
    
    vis_thread = threading.Thread(target=start_visualiser, 
                                  args=[board_address, tag, pop_label, xr, yr, xb, yb, key_conn])
    
    vis_thread.start()

    
def start_visualiser(board_address, tag, pop_label, xr, yr, xb=8, yb=8, key_conn=None):
        
    print("Calling \'start_visualiser\'")

    # Create visualiser
    # set_matplotlib_formats('png')
    vis = Visualiser(machine_address=board_address, tag=tag, x_factor=2, y_factor=2,
                     x_bits=xb, y_bits=yb)
    print("\nDisplaying visualiser")
    # Still testing whether it's possible to open the visualiser in a new cell
    # display(Javascript("Jupyter.notebook.execute_cells_below()"))
    vis.show()
    print("...awaiting game signals")

    refresh_time = 0.001
    while True:
#        print("updating...")
        score = vis._update(None)
#         plt.draw()  # "draw" is done in vis._update 
#         plt.pause(0.001)  # pause doesn't work, so use time.sleep(...)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        time.sleep(refresh_time)
        
    print("visualiser gets to here?")


# ----------------------------------------------------------------------------------------------------------------------
# Initialise Simulation and Parameters
# ----------------------------------------------------------------------------------------------------------------------

breakout = AutomatedBreakout()

# ----------------------------------------------------------------------------------------------------------------------
# Configure Visualiser
# ----------------------------------------------------------------------------------------------------------------------
key_input_connection = p.external_devices.SpynnakerLiveSpikesConnection(
    send_labels=[breakout.key_input.label], local_port=None)

print("\nRegister visualiser process")
key_input_connection.add_database_callback(functools.partial(
    start_vis_thread, pop_label=breakout.breakout_pop.label,
    xr=X_SCALE, yr=Y_SCALE,
    xb=np.uint32(np.ceil(np.log2(X_RES / X_SCALE))),
    yb=np.uint32(np.ceil(np.log2(Y_RES / Y_SCALE))),
    key_conn=key_input_connection))

p.external_devices.add_database_socket_address(
    "localhost", key_input_connection.local_port, None)

# ----------------------------------------------------------------------------------------------------------------------
# Run Simulation
# ----------------------------------------------------------------------------------------------------------------------

runtime = 1000 * 120
simulator = get_simulator()
print("\nLet\'s play breakout!")
p.run(runtime)

# End simulation
p.end()
print("Simulation Complete")


2021-10-06 09:13:29 WARNING: Population paddle_pop is missing spike data in region 3 from the following cores: (4, 5, 16); 
Getting spikes for ball_pop
|0%                          50%                         100%|
2021-10-06 09:13:29 WARNING: Population ball_pop is missing spike data in region 3 from the following cores: (3, 4, 16); 
Getting spikes for decision_input_pop
|0%                          50%                         100%|
Getting gsyn_exc for receive_rew_pop
|0%                          50%                         100%|


Scores: [(1,), (0,), (0,), (0,), (1,), (0,), (0,), (1,), (2,), (3,), (2,), (2,), (2,), (3,), (2,), (2,), (2,), (2,), (3,), (-2,), (-2,), (-2,), (-1,), (-1,), (-1,), (-1,), (-1,), (-1,), (-1,), (-1,), (-2,), (-2,), (-2,), (-2,), (-2,), (-3,), (-3,), (-3,), (-3,), (-4,), (-4,), (-4,), (-4,), (-4,), (-9,), (-9,), (-9,), (-9,), (-10,), (-10,), (-10,), (-10,), (-10,), (-11,), (-11,), (-11,), (-11,), (-12,), (-12,), (-12,), (-12,), (-12,), (-13,), (-13,), (-13,), (-13,), (-18,), (-18,), (-18,), (-18,), (-18,), (-19,), (-19,), (-19,), (-19,), (-20,), (-20,), (-20,), (-20,), (-20,), (-21,), (-21,), (-21,), (-21,), (-22,), (-22,), (-22,), (-22,), (-22,), (-27,), (-27,), (-27,), (-27,), (-28,), (-28,), (-28,), (-28,), (-28,), (-29,), (-29,), (-29,), (-29,), (-30,), (-30,), (-30,), (-30,), (-30,), (-31,), (-31,), (-31,), (-31,), (-36,), (-36,), (-36,), (-36,), (-36,), (-37,), (-37,), (-37,)]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Simulation Complete
